In [ ]:
import psycopg2
import pandas as pd

try:
    conn = psycopg2.connect(
        dbname="project", 
        user="postgres",       
        password="123",   
        host="localhost",          
        port="5432"                
    )
    print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")
    
cur = conn.cursor()

In [ ]:
add_constraints_and_indexes = """
ALTER TABLE IF EXISTS Orders             ADD COLUMN IF NOT EXISTS total_amount NUMERIC(12,2) DEFAULT 0;
ALTER TABLE IF EXISTS Sales_Transactions ADD COLUMN IF NOT EXISTS total_amount NUMERIC(12,2) DEFAULT 0;

ALTER TABLE IF EXISTS Inventory          ADD CONSTRAINT uq_inventory_store_product         UNIQUE (store_id, product_id);
ALTER TABLE IF EXISTS Transaction_Items  ADD CONSTRAINT uq_ti_tx_product                   UNIQUE (transaction_id, product_id);
ALTER TABLE IF EXISTS Order_Items        ADD CONSTRAINT uq_oi_order_product                UNIQUE (order_id, product_id);
ALTER TABLE IF EXISTS Vendor_Products    ADD CONSTRAINT uq_vp_vendor_product               UNIQUE (vendor_id, product_id);

ALTER TABLE IF EXISTS Products           ADD CONSTRAINT chk_products_unit_price_nonneg     CHECK (unit_price >= 0);
ALTER TABLE IF EXISTS Inventory          ADD CONSTRAINT chk_inventory_qty_nonneg           CHECK (quantity >= 0);
ALTER TABLE IF EXISTS Transaction_Items  ADD CONSTRAINT chk_ti_qty_pos                     CHECK (quantity > 0);
ALTER TABLE IF EXISTS Transaction_Items  ADD CONSTRAINT chk_ti_price_nonneg                CHECK (unit_price >= 0);
ALTER TABLE IF EXISTS Order_Items        ADD CONSTRAINT chk_oi_qty_pos                     CHECK (quantity > 0);
ALTER TABLE IF EXISTS Order_Items        ADD CONSTRAINT chk_oi_price_nonneg                CHECK (unit_price >= 0);
ALTER TABLE IF EXISTS Promotions         ADD CONSTRAINT chk_promos_date_order              CHECK (end_date >= start_date);
ALTER TABLE IF EXISTS Shifts             ADD CONSTRAINT chk_shifts_time_order              CHECK (end_time > start_time);
ALTER TABLE IF EXISTS Payments           ADD CONSTRAINT chk_payments_amount_nonneg         CHECK (amount >= 0);

CREATE INDEX IF NOT EXISTS idx_st_store_date     ON Sales_Transactions (store_id, transaction_date);
CREATE INDEX IF NOT EXISTS idx_st_customer       ON Sales_Transactions (customer_id);
CREATE INDEX IF NOT EXISTS idx_st_employee       ON Sales_Transactions (employee_id);
CREATE INDEX IF NOT EXISTS idx_st_date           ON Sales_Transactions (transaction_date);

CREATE INDEX IF NOT EXISTS idx_ti_tx             ON Transaction_Items (transaction_id);
CREATE INDEX IF NOT EXISTS idx_ti_product        ON Transaction_Items (product_id);

CREATE INDEX IF NOT EXISTS idx_orders_vendor_date ON Orders (vendor_id, order_date);
CREATE INDEX IF NOT EXISTS idx_orders_status      ON Orders (status);
CREATE INDEX IF NOT EXISTS idx_oi_order           ON Order_Items (order_id);
CREATE INDEX IF NOT EXISTS idx_oi_product         ON Order_Items (product_id);

CREATE INDEX IF NOT EXISTS idx_deliveries_order  ON Deliveries (order_id);
CREATE INDEX IF NOT EXISTS idx_deliveries_store  ON Deliveries (store_id);
CREATE INDEX IF NOT EXISTS idx_deliveries_time   ON Deliveries (delivered_at);

CREATE INDEX IF NOT EXISTS idx_inv_store         ON Inventory (store_id);
CREATE INDEX IF NOT EXISTS idx_inv_product       ON Inventory (product_id);

CREATE INDEX IF NOT EXISTS idx_employees_store   ON Employees (store_id);
CREATE INDEX IF NOT EXISTS idx_products_category ON Products (category);
CREATE INDEX IF NOT EXISTS idx_vendor_products_v ON Vendor_Products (vendor_id);
CREATE INDEX IF NOT EXISTS idx_vendor_products_p ON Vendor_Products (product_id);
CREATE INDEX IF NOT EXISTS idx_promotions_prod   ON Promotions (product_id);
CREATE INDEX IF NOT EXISTS idx_promotions_dates  ON Promotions (start_date, end_date);
CREATE INDEX IF NOT EXISTS idx_payments_tx       ON Payments (transaction_id);
CREATE INDEX IF NOT EXISTS idx_payments_date     ON Payments (payment_date);
CREATE INDEX IF NOT EXISTS idx_payments_method   ON Payments (payment_method);
"""

cur.execute(add_constraints_and_indexes)
conn.commit()
print("Constraints & indexes installed.")
